##Random Forest

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load and Clean
df = pd.read_csv('eeg_eye_state.csv')
X = df.drop('eyeDetection', axis=1)
y = df['eyeDetection']

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Result
predictions = model.predict(X_test)
print(f"Project Accuracy: {accuracy_score(y_test, predictions):.2%}")

Project Accuracy: 93.09%


##Neural Network project

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# 1. Load the cleaned data
df = pd.read_csv('cleaned_eeg_eye_state.csv')
X = df.drop('eyeDetection', axis=1)
y = df['eyeDetection']

# 2. Split and Scale (Scale is mandatory for NNs)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. Define the Neural Network
mlp = MLPClassifier(hidden_layer_sizes=(64, 32),
                    activation='relu',
                    solver='adam',
                    max_iter=500,
                    random_state=42)

# 4. Train and Evaluate
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

print(f"Neural Network Accuracy: {accuracy_score(y_test, y_pred):.2%}")

Neural Network Accuracy: 95.90%


In [4]:
!pip install tensorflow

##CNN

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# --- 1. DOWNLOAD & PATH SETUP ---
import kagglehub
path = kagglehub.dataset_download("tauilabdelilah/mrl-eye-dataset")
print("Dataset location:", path)

# Typically, MRL datasets require some folder organization.
# We assume the images are in subfolders 'open' and 'closed'
# within the downloaded path.
# IMG_SIZE = 64 # Resizing to 64x64 for efficiency

# --- 2. DATA AUGMENTATION ---
# This helps the model generalize to different eye shapes and lighting
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='training',
    color_mode='grayscale'
)

val_gen = datagen.flow_from_directory(
    path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    color_mode='grayscale'
)

# --- 3. CNN ARCHITECTURE ---

model = models.Sequential([
    # First Convolutional Block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),

    # Second Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Third Convolutional Block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Classification Head
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5), # Regularization to prevent overfitting
    layers.Dense(1, activation='sigmoid') # Binary Output: Open vs Closed
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# --- 4. TRAINING ---
history = model.fit(train_gen, epochs=10, validation_data=val_gen)

# --- 5. SAVE MODEL ---
model.save('eye_state_cnn.h5')

Using Colab cache for faster access to the 'mrl-eye-dataset' dataset.
Dataset location: /kaggle/input/mrl-eye-dataset
Found 67919 images belonging to 1 classes.
Found 16979 images belonging to 1 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 301s 139ms/step - accuracy: 0.9984 - loss: 0.0068 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 116s 54ms/step - accuracy: 1.0000 - loss: 1.0106e-12 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 116s 55ms/step - accuracy: 1.0000 - loss: 6.4062e-11 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 116s 54ms/step - accuracy: 1.0000 - loss: 5.5196e-12 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 116s 54ms/step - accuracy: 1.0000 - loss: 4.1353e-12 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 123s 58ms/step - accuracy: 1.0000 - loss: 3.3647e-13 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
2123/2123 ━━━━━━━━━━━━━━━━━━━━ 116s 55ms/step - accuracy: 1.0000 - loss: 1.3097e-10 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
2123/

In [3]:
!pip install tensorflow